In [ ]:
from src.model.llama import get_model_and_tokenizer
import json
from PIL import Image
import torch

In [ ]:
model_args = {}
with open("configs/Specialtokens/default.json") as f:
    special_token_map = json.load(f)
additional_tokens_dict = {x['type']: x['token'] for x in special_token_map['added_tokens']}

In [ ]:
model, tokenizer, _ = get_model_and_tokenizer(model_args, additional_tokens_dict, load_vision_model=True)

In [ ]:
model = model.to('cuda')

In [ ]:
image = Image.open("test.png")
prompt = "Here's an image:<IMGPLH> Create<IMGPLH> a caption for this image."

In [ ]:
model = model.to(torch.float16)

In [ ]:
inputs = model.prepare_input_ids_for_generation([prompt, prompt], [image, image, image, image], tokenizer)

In [ ]:
output = model.generate(input_ids=inputs['input_ids'].to("cuda"), attention_mask=inputs['attention_mask'].to("cuda"), encoded_image = inputs["encoded_image"], max_length=512, no_repeat_ngram_size=2, do_sample=True)

In [ ]:
tokenizer.batch_decode(output)

In [ ]:
model.push_to_hub("anananan116/TinyVLM")

In [ ]:
tokenizer.push_to_hub("anananan116/TinyVLM")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained("anananan116/TinyVLM", trust_remote_code = True).half().to('cuda').eval()
tokenizer = AutoTokenizer.from_pretrained("anananan116/TinyVLM")

In [ ]:
image = Image.open("test.png")
prompt = "Here's an image:<IMGPLH> Create a caption for this image."
inputs = model.prepare_input_ids_for_generation([prompt], [image], tokenizer)
outputs = model.generate(input_ids=inputs['input_ids'].to("cuda"), attention_mask=inputs['attention_mask'].to("cuda"), encoded_image = inputs["encoded_image"], max_new_tokens=128, do_sample=True)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)